Last Updated: 2022/11/17

By: Becky Yu

# Install and Import Packages

Uncomment and run the cell below to install packages

In [7]:
# !pip install webdriver_manager

In [8]:
import requests
import pandas as pd
import csv
import time

from cmath import e

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# Obtain Data From Sources

### 1. Wikipedia (Functions)

In [11]:
'''
Function to get wikipedia link from google.
'''
def get_wiki_link_from_google(chem_name):
    keyword = chem_name
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')

    driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)
    driver.get('https://www.google.com/')

    search_query = driver.find_element('name','q')
    search_query.send_keys(keyword + ' wikipedia')
    search_query.send_keys(Keys.RETURN)

    google_search_list = driver.find_element('id','search').text.split("\n")

    wiki_link = ""
    for i,line in enumerate(google_search_list):
        if 'Wikipedia' in line:
            wiki_link = google_search_list[i+1]
            break
    final_wiki_link = wiki_link.replace(" › ","/")
    return final_wiki_link

In [12]:
'''
Function to get one property from wikipedia from a given chemical name.
'''
def get_property(chemical):
    try:
        refined_chem_name = chemical.split()[0].capitalize() + "_".join([x for x in chemical.split()[1:]])
        try:
            df_pandas = pd.read_html("https://en.wikipedia.org/wiki/" + refined_chem_name, attrs = {'class': 'infobox ib-chembox'},  flavor='bs4', thousands ='.')
        except:
            link = get_wiki_link_from_google(chemical)
            df_pandas = pd.read_html(link, attrs = {'class': 'infobox ib-chembox'},  flavor='bs4', thousands ='.')
            
        all_df = pd.DataFrame(df_pandas[0])
        property_index = all_df[all_df[0] == 'Properties'].index[0]
        property_df = all_df.loc[property_index:]
        property_dict = dict(zip(property_df[0],property_df[1]))
        property_dict['Chemical Name'] = chemical
    except:
        property_dict = {'Chemical Name':chemical}
        
    return property_dict

In [13]:
'''
Function to get all properties from wikipedia from an input df with column 'Chemical Name'
'''
def get_properties(ingredients_df):
    all_chem_property_dicts = []
    for chemical in ingredients_df['Chemical Name']:
        property_dict = get_property(chemical)
        all_chem_property_dicts.append(property_dict)
    final_df = pd.DataFrame(all_chem_property_dicts)
    return final_df

In [14]:
'''
Function to get functionality from wikipedia from a given chemical name.
'''
def get_functionality(chemical,func_df):
    try:
        refined_chem_name = chemical.split()[0].capitalize() + "_".join([x for x in chemical.split()[1:]])
        response = requests.get(url="https://en.wikipedia.org/wiki/" + refined_chem_name)
        if response.status_code!=200:
            link = get_wiki_link_from_google(chemical)
            response = requests.get(url=link)
        soup = BeautifulSoup(response.content, 'html.parser')
        span = soup.find('span', {'id': 'Uses'})
        results = span.parent.find_next_siblings('p')
        
        result_subheadings_text = span.parent.find_next_siblings('p')
        all_uses_text = [x.text for x in results+result_subheadings_text]
        uses_text = " ".join(all_uses_text)
        all_funcs = []
        for func in func_df['Functionality']:
            if func in uses_text.lower():
                all_funcs.append(func)
        return all_funcs
    except:
        return []

In [15]:
'''
Function to get all functionalities from wikipedia from an input df with column 'Chemical Name'
'''
def get_functionalities(ingredients_df,functionality_file_name):
    all_chem_func_dict = {}
    func_df = pd.read_csv(functionality_file_name)
    for chemical in ingredients_df['Chemical Name']:
        all_chem_func_dict[chemical] = get_functionality(chemical,func_df)
    final_func_df = pd.DataFrame(all_chem_func_dict)
    return final_func_df

## 2. PubChem (Functions)

## 3. ChemBK (Functions)

In [ ]:
# Setup Selenium web driver

def setupDriver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get("https://www.chembk.com/en")
    return driver

In [ ]:
# Enter searchbox

def search(index, driver):
    # The first one searched from home page, the rest searched from result page
    if index == 0:
        searchBox = driver.find_element(By.ID, "chem")
    else:
        driver.find_element(By.ID, "chem").clear()
        searchBox = driver.find_element(By.ID, "chem")

    searchBox.send_keys(chemical_name)
    searchBox.send_keys(Keys.ENTER)

In [ ]:
# Mark properties as empty when no results found

def writeEmpty(writer, index, chemical_name):
    writer.writerow([index, chemical_name, "", "", "", "", "", "", "", ""])

In [ ]:
# Extract information from Supplier Page

def extractSupplierPage(tds, driver, writer):
    for row in driver.find_elements(By.TAG_NAME, 'tr'):
        tds = row.find_elements(By.TAG_NAME, 'td')

        # Get formula
        if tds[0].text == "Chemical Formula":
            formula = tds[1].text

        # Get molecular weight
        if tds[0].text == "Molecular Weight":
            weight = tds[1].text
            break
        
    writer.writerow([index, chemical_name, "", water_solubility, weight, density, formula, "", synonyms, ""])
        
    driver.get("https://www.chembk.com/en")

## Main

In [20]:
# Main - Wikipedia
if __name__ == '__main__':

#     functionality_file_name = 'All_functionalities.csv'  #original
    functionality_file_name = 'Wikipedia_AllFunctionalities.csv'
    
#     ingredients_df = pd.read_excel("cosing_clean (1).xlsx") #original
    ingredients_df = pd.read_excel("cosing_clean.xlsx")
    
    properties_df = get_properties(ingredients_df.iloc[0:3])

    functionalites_df = get_functionalities(ingredients_df.iloc,functionality_file_name)

#     properties_df.to_csv("All_wiki_properties.csv")
    
    # Write properties to the final integrated file "output_mmdd.csv"
    properties_df.to_csv("/output/output_1117.csv")
    functionalites_df.to_csv("All_wiki_functionalities.csv")

C:\Users\Becky\AppData\Local\Temp\ipykernel_3064\1223384760.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)


TypeError: Cannot index by location index with a non-integer key

In [ ]:
# Main - ChemBK
if __name__ == '__main__':

    with open('chembk_1117_all.csv', 'w', encoding = 'UTF8', newline="") as f:
        try:
            header = ['id', 'Chemical Name', 'LogP (Partition coefficient)', 'LogS (water solubility of the ingredient)', 
                      'Molecular weight', 'Density', 'Chemical formula', 'Structure', 'Synonyms', 'Functionalities']
            writer = csv.writer(f)
            writer.writerow(header)

            # Setup Selenium web driver
            driver = setupDriver()

            count_e = 0
            for index, row in df.iterrows():
                try:
                    found = False

                    # Use chemical name as search keyword
                    # Remove slash in the string which cannot be put into search box
                    chemical_name = row['Chemical Name'].title().replace("/", " ")

                    # Initialization
                    macthed_name = ""
                    synonyms = ""
                    formula = ""
                    weight = ""
                    density = ""
                    water_solubility = ""

                    # Search
                    search(index, driver)

                    # -- Choose which page to move on -- 

                    trs = driver.find_elements(By.TAG_NAME, 'tr')
                    tds = driver.find_elements(By.TAG_NAME, 'td')

                    # If no table -> No Results found
                    if len(trs) == 0 or len(tds) == 0:   
                        writeEmpty(writer, index, chemical_name)
                        print(chemical_name + " : no results found.")
                        continue

                    # If exact name match found
                    if len(driver.find_elements(By.XPATH, '//a[@href="/en/chem/'+chemical_name+'"]')) == 1:
                        matched_name = chemical_name
                        nextPage = driver.find_element(By.XPATH, '//a[@href="/en/chem/'+chemical_name+'"]')
                        found = True
                        print(chemical_name + " : exact name found.")

                    # If no exact match, having multiple results:
                    #    1. See if the search name matches one of the Synonyms
                    if found == False:
                        tbodys = driver.find_elements(By.TAG_NAME, 'tbody')
                        trs = tbodys[0].find_elements(By.TAG_NAME, 'tr')
                        for tr in trs:
                            cols = tr.find_elements(By.TAG_NAME, 'td')
                            if len(cols) >= 5:   # make sure it is data, not some random row with tds
                                if chemical_name in cols[3].text:
                                    nextPage = cols[2].find_element(By.TAG_NAME, 'a')
                                    matched_name = cols[2].text
                                    found = True
                                    print(chemical_name + " : have matched synonym.")
                                    break

                    #    2. In the rest options, pick the first one with Molecular Formula shown
                    if found == False:
                        trs = driver.find_elements(By.TAG_NAME, 'tr')
                        for tr in trs:
                            cols = tr.find_elements(By.TAG_NAME, 'td')
                            if len(cols) >= 5:   # make sure it is data, not some random row with tds
                                if cols[4].text != "":
                                    nextPage = cols[2].find_element(By.TAG_NAME, 'a')
                                    matched_name = cols[2].text
                                    found = True
                                    print(chemical_name + " : pick the first one with molecular.")
                                    break

                    #    3. If no options showing Molecular Formula, pick the first one containing the chemical name as its substring of the results
                    if found == False:
                        if driver.find_elements(By.XPATH, '//a[contains(@href, "%s")]' % chemical_name):
                            options = driver.find_elements(By.XPATH, '//a[contains(@href, "%s")]' % chemical_name)
                            if len(options) > 1:
                                matched_name = options[1].text
                                nextPage = options[1]   # Skip option 0
                                found = True
                                print(chemical_name + " : pick the first one containing the chemical name.")
                                break

                    #  4. Pick the first one if no conditions matched.
                    if found == False:
                        tbodys = driver.find_elements(By.TAG_NAME, 'tbody')
                        if len(tbodys) != 0:
                            for tbody in tbodys:
                                trs = tbody.find_elements(By.TAG_NAME, 'tr')
                                if len(trs) != 0:                    
                                    tds = trs[0].find_elements(By.TAG_NAME, 'td')
                                    nextPage = tds[2].find_element(By.TAG_NAME, 'a')
                                    matched_name = tds[2].text
                                    print(chemical_name + " : pick the first one.")
                                    break

                    # If chemical found
                    nextPage.click()

                    # If the page found is a supplier page
                    if len(driver.find_elements(By.TAG_NAME, 'h4')) > 0:
                        if driver.find_elements(By.TAG_NAME, 'h4')[0].text == "Request for quotation":
                            extractSupplierPage(tds, driver, writer)
                            continue

                    # Extract info from header paragraph
                    formula = driver.find_elements(By.TAG_NAME, 'h4')[2].text.split(":")[1].strip()
                    weight =  driver.find_elements(By.TAG_NAME, 'h4')[3].text.split(":")[1].strip()

                    # Extract info from table
                    for row in driver.find_elements(By.TAG_NAME, 'tr'):
                        tds = row.find_elements(By.TAG_NAME, 'td')

                        # Grab Synonyms
                        if tds[0].text == "Synonyms":
                            syns = tds[1].find_elements(By.TAG_NAME, 'a')
                            for syn in syns:
                                if len(syn.text.split(",")) > 1:
                                    synonyms += syn.text + ","
                                else:
                                    synonyms += syn.text

                        # Grab Density
                        if tds[0].text == "Density":
                            density = tds[1].text

                        # Grab Water Solubility
                        if tds[0].text == "Water Solubility":
                            water_solubility = tds[1].text
                            break   # exit earlier if all the properties have found

                    writer.writerow([index, chemical_name, "", water_solubility, weight, density, formula, "", synonyms, ""])
                except Exception as e:
                    writer.writerow([index, chemical_name, "", "", "", "", "", "", ""])
                    count_e += 1
                    print("error" + str(count_e) + ": " + chemical_name + " - " + str(e))
                    continue
        except Exception as e:
            print(e)